In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from dataiku.scenario import Trigger


# Accessing the DSS Global/Project Variables
proj = dataiku.Project()
variables = proj.get_variables()

In [10]:
dpc_dataset = dataiku.Dataset("dpc_covid19_prepared_windows_by_ntile10")


In [14]:
df = dpc_dataset.get_dataframe()

/Users/terrijohn/Library/dss/design/code-envs/python/python37_holidays/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [15]:
df.head()

,ntile10,nuovi_positivi_min,nuovi_positivi_max,nuovi_positivi_avg,nuovi_positivi_sum,nuovi_positivi_stddev,count
0,1,78,451,253.000000,21252,84.220022,84
1,2,463,1397,918.452381,77150,294.270362,84
2,3,1400,2818,2035.096386,168913,456.517374,83
3,4,2834,4717,3713.726190,311953,554.433464,84
4,5,4720,7537,5963.337349,494957,817.652847,83


In [20]:
int(df['nuovi_positivi_min'][df['ntile10'] == 1])

78

In [21]:
range(int(df['nuovi_positivi_min'][df['ntile10'] == 1]), int(df['nuovi_positivi_max'][df['ntile10'] == 1]))

range(78, 451)

In [27]:
for num in range(1,11):
    print(range(int(df['nuovi_positivi_min'][df['ntile10'] == num]), int(df['nuovi_positivi_max'][df['ntile10'] == n])))

1
2
3
4
5
6
7
8
9
10


In [22]:
variables["standard"]['n1'] = range(int(df['nuovi_positivi_min'][df['ntile10'] == 1]), int(df['nuovi_positivi_max'][df['ntile10'] == 1]))

In [23]:
variables["standard"]['n1']

range(78, 451)

In [25]:
79 in variables["standard"]['n1']

True

In [0]:
# Create a list of currently existing global/project variables to test if our length_list has been set.
global_check = list(variables['standard'].keys())

In [0]:
# Testing to see if the length_list global/project variable exists.
# If it does not, it will be created.
if 'length_list' not in global_check:
    length_list = len(file_list)
    variables['standard'] = {"length_list" : length_list}
    proj.set_variables(variables)

# If it does exist, the code continues running to the next step.
else:
    continue

In [0]:
# Create variables that we'll use to test the file list length.
current_length_list = len(file_list)
var_length_list = int(dataiku.get_custom_variables().get("length_list"))
t = Trigger()

In [0]:
# For loop to check if the trigger file name already exists in the Managed folder.
# Once the trigger file is added (in this example "trigger_file.txt"),
# the count variables keep the scenario from running indefinitely.
for file in file_list:
    if current_length_list > var_length_list and file == '/trigger_file.txt':
        t.fire()

In [0]:
# Now we reset the results as DSS Global/Project variables.
variables['standard'] = {"length_list" : current_length_list}
proj.set_variables(variables)